In [1]:
#!/usr/bin/env python3
import sys
import copy
sys.path.append('../')

from __future__ import print_function
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.theory import theoryPrediction
from smodels.experiment.databaseObj import Database, ExpResultList
from smodels.tools import coverage
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.printers.masterPrinter import MPrinter
from smodels.tools import ioObjects
import time
import os
from collections import OrderedDict
from configparser import ConfigParser, NoSectionError, NoOptionError
setLogLevel("warning")

In [2]:
# Set the path to the database
database = Database(os.path.expanduser('../test/database'))
databaseVersion = database.databaseVersion

In [3]:
parameterFile='../test/testPrinters_parameters.ini'
parser = ConfigParser(inline_comment_prefixes=(';',))
ret = parser.read(parameterFile)
parser.set("printer","outputType","python")
parser.set("printer","outputFormat","current")

inputFile = '../test/testFiles/slha/lightEWinos.slha'
outputDir = '../test/'

In [4]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
minmassgap = parser.getfloat("parameters", "minmassgap") * GeV

masterPrinter = MPrinter()
masterPrinter.setPrinterOptions(parser)
masterPrinter.setOutPutFiles(os.path.join(
    outputDir, os.path.basename(inputFile)))
# for printer in masterPrinter.Printers.values():
#     printer.output = 'stdout'


In [5]:
listOfExpRes = database.getExpResults()
masterPrinter.addObj(ExpResultList(listOfExpRes))

inputStatus = ioObjects.FileStatus()
if parser.getboolean("options", "checkInput"):
    inputStatus.checkFile(inputFile)

printParameters = []
if parser.has_section('parameters'):
    printParameters += list(parser.items('parameters'))
if parser.has_section('particles'):
    printParameters += list(parser.items('particles'))
if parser.has_section('options'):
    printParameters += list(parser.items('options'))

printParameters = OrderedDict(printParameters)
outputStatus = ioObjects.OutputStatus(inputStatus.status, inputFile,
                                      printParameters, databaseVersion)
masterPrinter.addObj(outputStatus)

In [6]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
promptWidth = None
stableWidth = None
if parser.has_option("particles", "promptWidth"):
    promptWidth = parser.getfloat("particles", "promptWidth")*GeV
if parser.has_option("particles", "stableWidth"):
    stableWidth = parser.getfloat("particles", "stableWidth")*GeV
model.updateParticles(inputFile=inputFile,
                      promptWidth=promptWidth, stableWidth=stableWidth)

In [7]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
smstoplist = decomposer.decompose(model, sigmacut,
                                  massCompress=parser.getboolean(
                                      "options", "doCompress"),
                                  invisibleCompress=parser.getboolean(
                                      "options", "doInvisible"),
                                  minmassgap=minmassgap)


In [8]:
masterPrinter.addObj(smstoplist)

In [9]:
allPredictions = []
combineResults = False
useBest = True
combineResults = parser.getboolean("options", "combineSRs")
allSRs = parser.getboolean("options", "reportAllSRs")
if allSRs:  # If set print out all SRs and skip combination
    useBest = False
    combineResults = False

for expResult in listOfExpRes:
    theorypredictions = theoryPredictionsFor(expResult, smstoplist,
                                             useBestDataset=useBest, combinedResults=combineResults,
                                             marginalize=False)
    if not theorypredictions:
        continue
    allPredictions += theorypredictions._theoryPredictions

if parser.getboolean("options", "computeStatistics"):
    for theoPred in allPredictions:
        theoPred.computeStatistics()

maxcond = parser.getfloat("parameters", "maxcond")
theoryPredictions = theoryPrediction.TheoryPredictionList(
    allPredictions, maxcond)


In [10]:
outputStatus.updateStatus(1)
masterPrinter.addObj(theoryPredictions)

In [11]:
if parser.getboolean("options", "testCoverage"):
    """ Testing coverage of model point, add results to the output file """
    if parser.has_option("options", "coverageSqrts"):
        sqrts = parser.getfloat("options", "coverageSqrts")*TeV
    else:
        sqrts = None
    uncovered = coverage.Uncovered(
        smstoplist, sigmacut=sigmacut, sqrts=sqrts)
    masterPrinter.addObj(uncovered)


In [12]:
if parser.has_option("options", "combineAnas"):
    """ Combine analyses """

    combineAnas = parser.get("options", "combineAnas").replace(" ","").split(",")
    if combineAnas:
        if combineResults is True:
            logger.warning("Combining analyses with signal region combination (combineSRs=True) might significantly reduce CPU performance.")
        combiner = TheoryPredictionsCombiner.selectResultsFrom(theoryPredictions,
                                                               combineAnas)
        # Only compute combination if at least one result was selected
        if combiner is not None:
            combiner.computeStatistics()
            masterPrinter.addObj(combiner)


In [13]:
masterCopy = copy.deepcopy(masterPrinter)
x = masterPrinter.flush()['python']
os.remove(masterPrinter.Printers['python'].filename)

In [14]:
sortedKeys = sorted(x.keys())
for key in sortedKeys:
    print(key)

CombinedRes
ExptRes
OutputStatus
SMS Decomposition
Total xsec for missing topologies (fb)
Total xsec for missing topologies with displaced decays (fb)
Total xsec for missing topologies with prompt decays (fb)
Total xsec for topologies outside the grid (fb)
missing topologies
missing topologies with displaced decays
missing topologies with prompt decays
topologies outside the grid


In [15]:
for v in x['CombinedRes']:
    print(v)

{'AnalysisID': 'ATLAS-CONF-2013-037', 'r': 0.1092856, 'r_expected': 0.1009566, 'likelihood': 0.01900539, 'l_max': 0.02067189, 'l_SM': 0.02067189}


In [16]:
for v in x['ExptRes']:
    print(v)

{'maxcond': 0.0, 'theory prediction (fb)': 0.04371422, 'upper limit (fb)': 0.4, 'expected upper limit (fb)': 0.433, 'TxNames': ['T1bbtt'], 'Mass (GeV)': None, 'AnalysisID': 'ATLAS-CONF-2013-037', 'DataSetID': 'SRbC3', 'AnalysisSqrts (TeV)': 8.0, 'lumi (fb-1)': 20.7, 'dataType': 'efficiencyMap', 'r': 0.1092856, 'r_expected': 0.1009566, 'TxNames weights (fb)': {'T1bbtt': 0.04371422206454377}, 'likelihood': 0.01900539, 'l_max': 0.02067189, 'l_SM': 0.02067189}
{'maxcond': 0.0, 'theory prediction (fb)': 1530.012, 'upper limit (fb)': 23325110.0, 'expected upper limit (fb)': None, 'TxNames': ['T1'], 'Mass (GeV)': None, 'AnalysisID': 'ATLAS-SUSY-2013-02', 'DataSetID': None, 'AnalysisSqrts (TeV)': 8.0, 'lumi (fb-1)': 20.3, 'dataType': 'upperLimit', 'r': 6.559505e-05, 'r_expected': None, 'TxNames weights (fb)': {'T1': 1530.0117493459818}}


In [17]:
print(x['OutputStatus'])

{'sigmacut': 10.0, 'minmassgap': 5.0, 'maxcond': 0.2, 'ncpus': 1, 'model': 'share.models.mssm', 'promptwidth': 1e-11, 'stablewidth': 1e-25, 'checkinput': True, 'doinvisible': True, 'docompress': True, 'computestatistics': True, 'testcoverage': True, 'combinesrs': False, 'combineanas': 'ATLAS-CONF-2013-037,CMS-SUS-13-012', 'reportallsrs': False, 'experimentalfeatures': False, 'file status': 1, 'decomposition status': 1, 'warnings': 'Input file ok', 'input file': '../test/testFiles/slha/lightEWinos.slha', 'database version': 'unittest220', 'smodels version': '2.2.1'}


In [18]:
print(len(x['SMS Decomposition']))
for sms in x['SMS Decomposition'][:3]:
    print(sms)

105
{'ID': 1, 'SMS': '(PV > N1,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,q)', 'Masses (GeV)': [('N1', 68.1), ('C1+/C1-', 134.4), ('N1/N1~', 68.1)], 'PIDs': [('N1', 1000022), ('C1+/C1-', [1000024, -1000024]), ('N1/N1~', [1000022, -1000022])], 'Weights (fb)': {'xsec 8.0 TeV': 38.5, 'xsec 13.0 TeV': 53.8}}
{'ID': 2, 'SMS': '(PV > N1,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,c)', 'Masses (GeV)': [('N1', 68.1), ('C1+/C1-', 134.4), ('N1/N1~', 68.1)], 'PIDs': [('N1', 1000022), ('C1+/C1-', [1000024, -1000024]), ('N1/N1~', [1000022, -1000022])], 'Weights (fb)': {'xsec 8.0 TeV': 38.5, 'xsec 13.0 TeV': 53.8}}
{'ID': 3, 'SMS': '(PV > inv,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,q)', 'Masses (GeV)': [('inv', 135.3), ('C1+/C1-', 134.4), ('N1/N1~', 68.1)], 'PIDs': [('inv', 1000023), ('C1+/C1-', [1000024, -1000024]), ('N1/N1~', [1000022, -1000022])], 'Weights (fb)': {'xsec 8.0 TeV': 64.7, 'xsec 13.0 TeV': 121.0}}


In [19]:
print(x['Total xsec for missing topologies (fb)'])
print(x['Total xsec for missing topologies with displaced decays (fb)'])
print(x['Total xsec for missing topologies with prompt decays (fb)'])
print(x['Total xsec for topologies outside the grid (fb)'])

5223.424
0.0
5223.424
47.18918


In [20]:
for v in x['missing topologies'][:2]:
    print(v)

{'sqrts (TeV)': 13.0, 'weight (fb)': 1203.688, 'SMS': '(PV > MET,MET,jet,jet,l,nu)', 'SMS IDs': [31, 36, 10, 18, 8, 9, 16, 17, 29, 30, 34, 35, 32, 37, 11, 19]}
{'sqrts (TeV)': 13.0, 'weight (fb)': 600.2543, 'SMS': '(PV > MET,MET,jet,jet,ta,nu)', 'SMS IDs': [41, 26, 24, 25, 39, 40, 42, 27]}


In [21]:
print(x['missing topologies with displaced decays'])

[]


In [22]:
for v in x['missing topologies with prompt decays'][:2]:
    print(v)

{'sqrts (TeV)': 13.0, 'weight (fb)': 1203.688, 'SMS': '(PV > MET,MET,jet,jet,l,nu)', 'SMS IDs': [31, 36, 10, 18, 8, 9, 16, 17, 29, 30, 34, 35, 32, 37, 11, 19]}
{'sqrts (TeV)': 13.0, 'weight (fb)': 600.2543, 'SMS': '(PV > MET,MET,jet,jet,ta,nu)', 'SMS IDs': [41, 26, 24, 25, 39, 40, 42, 27]}


In [23]:
for v in x['topologies outside the grid']:
    print(v)

{'sqrts (TeV)': 13.0, 'weight (fb)': 47.18918, 'SMS': '(PV > MET,MET,b,b,b,b)', 'SMS IDs': [62]}


In [24]:
masterCopy.outputFormat = "version2"
for printer in masterCopy.Printers.values():
    printer.outputFormat = "version2"
    
x = masterCopy.flush()['python']
os.remove(masterCopy.Printers['python'].filename)

In [25]:
sortedKeys = sorted(x.keys())
for key in sortedKeys:
    print(key)

CombinedRes
Element
ExptRes
OutputStatus
Total xsec for missing topologies (fb)
Total xsec for missing topologies with displaced decays (fb)
Total xsec for missing topologies with prompt decays (fb)
Total xsec for topologies outside the grid (fb)
missing topologies
missing topologies with displaced decays
missing topologies with prompt decays
topologies outside the grid


In [26]:
for v in x['CombinedRes']:
    print(v)

{'AnalysisID': 'ATLAS-CONF-2013-037', 'r': 0.1092856, 'r_expected': 0.1009566, 'likelihood': 0.01900539, 'l_max': 0.02067189, 'l_SM': 0.02067189}


In [27]:
for v in x['ExptRes']:
    print(v)

{'maxcond': 0.0, 'theory prediction (fb)': 0.04371422, 'upper limit (fb)': 0.4, 'expected upper limit (fb)': 0.433, 'TxNames': ['T1bbtt'], 'Mass (GeV)': None, 'AnalysisID': 'ATLAS-CONF-2013-037', 'DataSetID': 'SRbC3', 'AnalysisSqrts (TeV)': 8.0, 'lumi (fb-1)': 20.7, 'dataType': 'efficiencyMap', 'r': 0.1092856, 'r_expected': 0.1009566, 'TxNames weights (fb)': {'T1bbtt': 0.04371422206454377}, 'likelihood': 0.01900539, 'l_max': 0.02067189, 'l_SM': 0.02067189}
{'maxcond': 0.0, 'theory prediction (fb)': 1530.012, 'upper limit (fb)': 23325110.0, 'expected upper limit (fb)': None, 'TxNames': ['T1'], 'Mass (GeV)': None, 'AnalysisID': 'ATLAS-SUSY-2013-02', 'DataSetID': None, 'AnalysisSqrts (TeV)': 8.0, 'lumi (fb-1)': 20.3, 'dataType': 'upperLimit', 'r': 6.559505e-05, 'r_expected': None, 'TxNames weights (fb)': {'T1': 1530.0117493459818}}


In [28]:
print(x['OutputStatus'])

{'sigmacut': 10.0, 'minmassgap': 5.0, 'maxcond': 0.2, 'ncpus': 1, 'model': 'share.models.mssm', 'promptwidth': 1e-11, 'stablewidth': 1e-25, 'checkinput': True, 'doinvisible': True, 'docompress': True, 'computestatistics': True, 'testcoverage': True, 'combinesrs': False, 'combineanas': 'ATLAS-CONF-2013-037,CMS-SUS-13-012', 'reportallsrs': False, 'experimentalfeatures': False, 'file status': 1, 'decomposition status': 1, 'warnings': 'Input file ok', 'input file': '../test/testFiles/slha/lightEWinos.slha', 'database version': 'unittest220', 'smodels version': '2.2.1'}


In [29]:
print(len(x['Element']))
for sms in x['Element'][:3]:
    print(sms)

105
{'ID': 1, 'Particles': '[[],[[q,q]]]', 'final states': ['N1', 'N1/N1~'], 'Masses (GeV)': [[68.1], [134.4, 68.1]], 'PIDs': [[1000022], [[1000024, -1000024], [1000022, -1000022]]], 'Weights (fb)': {'xsec 8.0 TeV': 38.5, 'xsec 13.0 TeV': 53.8}}
{'ID': 2, 'Particles': '[[],[[q,c]]]', 'final states': ['N1', 'N1/N1~'], 'Masses (GeV)': [[68.1], [134.4, 68.1]], 'PIDs': [[1000022], [[1000024, -1000024], [1000022, -1000022]]], 'Weights (fb)': {'xsec 8.0 TeV': 38.5, 'xsec 13.0 TeV': 53.8}}
{'ID': 3, 'Particles': '[[],[[q,q]]]', 'final states': ['inv', 'N1/N1~'], 'Masses (GeV)': [[135.3], [134.4, 68.1]], 'PIDs': [[1000023], [[1000024, -1000024], [1000022, -1000022]]], 'Weights (fb)': {'xsec 8.0 TeV': 64.7, 'xsec 13.0 TeV': 121.0}}


In [30]:
print(x['Total xsec for missing topologies (fb)'])
print(x['Total xsec for missing topologies with displaced decays (fb)'])
print(x['Total xsec for missing topologies with prompt decays (fb)'])
print(x['Total xsec for topologies outside the grid (fb)'])

5223.424
0.0
5223.424
47.18918


In [31]:
for v in x['missing topologies'][:2]:
    print(v)

{'sqrts (TeV)': 13.0, 'weight (fb)': 1203.688, 'SMS': '[jet,jet,l,nu]  (MET,MET)', 'SMS IDs': [31, 36, 10, 18, 8, 9, 16, 17, 29, 30, 34, 35, 32, 37, 11, 19]}
{'sqrts (TeV)': 13.0, 'weight (fb)': 600.2543, 'SMS': '[jet,jet,ta,nu]  (MET,MET)', 'SMS IDs': [41, 26, 24, 25, 39, 40, 42, 27]}


In [32]:
print(x['missing topologies with displaced decays'])

[]


In [33]:
for v in x['missing topologies with prompt decays'][:2]:
    print(v)

{'sqrts (TeV)': 13.0, 'weight (fb)': 1203.688, 'SMS': '[jet,jet,l,nu]  (MET,MET)', 'SMS IDs': [31, 36, 10, 18, 8, 9, 16, 17, 29, 30, 34, 35, 32, 37, 11, 19]}
{'sqrts (TeV)': 13.0, 'weight (fb)': 600.2543, 'SMS': '[jet,jet,ta,nu]  (MET,MET)', 'SMS IDs': [41, 26, 24, 25, 39, 40, 42, 27]}


In [34]:
for v in x['topologies outside the grid']:
    print(v)

{'sqrts (TeV)': 13.0, 'weight (fb)': 47.18918, 'SMS': '[b,b,b,b]  (MET,MET)', 'SMS IDs': [62]}
